In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 19631266
paper_name = 'zhou_costa_2009' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/table.xlsx', sheet_name='table.csv', skiprows=2)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 253 x 6


In [10]:
original_data.head()

,Unnamed: 0,Yeast protein name,Yeast function,IC50,Human SwissProt number,Human function
0,YJL179W,PFD1,Protein with similarity to bovine prefoldin su...,0.2444,O60925,Prefoldin subunit 1
1,YLR113W,HOG1,MAP kinase (MAPK) central to the high-osmolari...,0.3059,Q16539,Mitogen-activated protein kinase 14
2,YJR090C,GRR1,F-box protein that targets G1 cyclins and Gic1...,0.3464,AAH07557,F-box/LRR-repeat protein 20
3,YDR448W,ADA2,Component of two nucleosomal histone acetyltra...,0.375,O75478,Transcriptional adapter 2-alpha
4,YBR278W,DPB3,DNA polymerase epsilon third subunit,0.4023,Q9NR33,DNA polymerase epsilon subunit 4


In [11]:
original_data['orf'] = original_data['Unnamed: 0'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [15]:
original_data.loc[original_data['orf']=='YAR002AW','orf'] = 'YAR002W-A'

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Yeast protein name, Yeast function, IC50, Human SwissProt number, Human function, orf]
Index: []


In [18]:
original_data.loc[original_data['IC50']=='Above 5','IC50'] = 5

In [20]:
original_data['data'] = pd.to_numeric(original_data['IC50'], errors='coerce')

In [21]:
original_data.set_index('orf', inplace=True)

In [22]:
original_data = original_data[['data']].copy()

In [23]:
original_data = original_data.groupby(original_data.index).mean()

In [24]:
original_data.shape

(252, 1)

In [25]:
# IC50 from paper
wt = 4.47 

original_data['data'] = original_data['data'] - wt

# Prepare the final dataset

In [26]:
data = original_data.copy()

In [27]:
dataset_ids = [115]
datasets = datasets.reindex(index=dataset_ids)

In [28]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [29]:
data.head()

dataset_id,115
data_type,value
orf,
YAL002W,-3.3680
YAL012W,-3.6233
YAL023C,-1.5380
YAL042C-A,-3.0750
YAL042W,-3.0670


## Subset to the genes currently in SGD

In [30]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [31]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,115
,data_type,value
gene_id,orf,
2,YAL002W,-3.3680
10,YAL012W,-3.6233
21,YAL023C,-1.5380
1865,YAL042C-A,-3.0750
40,YAL042W,-3.0670


# Normalize

In [32]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [33]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [34]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            115          
data_type           value    valuez
gene_id orf                        
2       YAL002W   -3.3680 -4.284113
10      YAL012W   -3.6233 -4.608702
21      YAL023C   -1.5380 -1.957449
1865    YAL042C-A -3.0750 -3.911593
40      YAL042W   -3.0670 -3.901421

# Print out

In [35]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [36]:
from IO.save_data_to_db3 import *

In [37]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 19631266...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Updating the data_modified_on field...
